<a href="https://colab.research.google.com/github/adarsh-meher/MachineHack---Predict-Delivery-Time/blob/master/MH_Predict_Delivery_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##### Installing required libraries
!pip install hyperopt
!pip install h2o
!pip install category_encoders
!pip install rgf-python
!pip install imblearn

     |████████████████████████████████| 123.4MB 96kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.1-py2.py3-none-any.whl size=123410554 sha256=1f1ea32a5f4b123163ff74e43dac5616e0a023cb5e5499ff10cb816869122be5
  Stored in directory: /root/.cache/pip/wheels/ff/82/f4/3fe9c895b93c53b25ae44dc44aa54dad571c711f48bc485fe9
Successfully built h2o
     |████████████████████████████████| 102kB 5.9MB/s 
     |████████████████████████████████| 768kB 8.8MB/s 


In [2]:
! git clone https://github.com/adarsh-meher/MachineHack---Predict-Delivery-Time.git

Cloning into 'MachineHack---Predict-Delivery-Time'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [0]:
import os
os.chdir('/content/MachineHack---Predict-Delivery-Time')

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgbm
import h2o
import hyperopt as hopt
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import nltk
import spacy as scy
import time

In [0]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.multiclass import OutputCodeClassifier,OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [0]:
train_data = pd.ExcelFile('Data_Train.xlsx').parse('Sheet1')
test_data = pd.ExcelFile('Data_Test.xlsx').parse('Sheet1')

In [7]:
train_data.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [0]:
#### Concatenating train and test data.
train_data['Type'] = ['train']*train_data.shape[0]
test_data['Type'] = ['test']*test_data.shape[0]
total_data = pd.concat([train_data.drop(['Delivery_Time'],axis = 1),test_data],axis = 0,ignore_index = True)

In [9]:
#### Creating list of all uniques cuisines across train + test data
total_data['Cuisines_List'] = total_data['Cuisines'].map(lambda x : [ a.strip() for a in x.split(',') ] )
list_cuisines = []
for x in total_data['Cuisines_List']:
  for t in x:
    list_cuisines.append(t)

uniq_cuisines = list(dict.fromkeys(list_cuisines))
print('Number of unique cuisines : %s' % len(uniq_cuisines))

Number of unique cuisines : 101


In [10]:
#### Creating columns of cuisines with 0/1 entry indicating the restaurant served the cuisine(1) or not (0)
start = time.time()
for c in uniq_cuisines:
  total_data[c] = total_data['Cuisines_List'].map(lambda x : 1 if c in x else 0)
end = time.time()
print(end-start)

0.536329984664917


In [0]:
def conv_str_to_float(x):
  if len(x) ==1:
    try:
      return int(x[0])
    except Exception as e:
      return 0.0
  else:
    t = ''.join(x[1].split(',')) if ',' in x[1] else x[1]
    return float(t)

total_data['Average_Cost_New'] = total_data['Average_Cost'].apply(lambda x : conv_str_to_float(x.split('₹')))
total_data['Minimum_Order_New'] = total_data['Minimum_Order'].apply(lambda x : conv_str_to_float(x.split('₹')))

In [0]:
def conv_str_to_float_2(x):
  try:
    return float(x)
  except Exception as e:
    return -1.0

total_data['Rating_New'] = total_data['Rating'].apply(lambda x : conv_str_to_float_2(x))
total_data['Votes_New'] = total_data['Votes'].apply(lambda x : conv_str_to_float_2(x))
total_data['Reviews_New'] = total_data['Reviews'].apply(lambda x : conv_str_to_float_2(x))

In [0]:
total_data = pd.get_dummies(total_data,columns = ['Location'])

In [14]:
train_new,test_new = total_data[total_data['Type'] == 'train'],total_data[total_data['Type'] == 'test']
train_new.index = range(train_new.shape[0])
test_new.index = range(test_new.shape[0])
print(train_new.shape)
print(test_new.shape)

(11094, 150)
(2774, 150)


In [0]:
from sklearn.preprocessing import LabelEncoder
le_fit = LabelEncoder().fit(train_data['Delivery_Time'])
target_enc = le_fit.transform(train_data['Delivery_Time'])
train_data['Delivery_Time_Enc'] = target_enc

In [16]:
train_data['Delivery_Time_Enc'].value_counts()

3    7406
4    2665
5     923
1      62
2      20
6      14
0       4
Name: Delivery_Time_Enc, dtype: int64

In [0]:
train_new1,val_new1,train_target,val_target = train_test_split(train_new,target_enc,test_size = 0.2,stratify = target_enc)

In [22]:
print(train_data['Delivery_Time_Enc'].value_counts())
print(pd.Series(val_target).value_counts())

3    7406
4    2665
5     923
1      62
2      20
6      14
0       4
Name: Delivery_Time_Enc, dtype: int64
3    1481
4     533
5     185
1      12
2       4
6       3
0       1
dtype: int64


In [18]:
del_time_dict = dict(train_data['Delivery_Time_Enc'].value_counts())
modal_class = train_data['Delivery_Time_Enc'].value_counts().argmax()
modal_class_value = train_data['Delivery_Time_Enc'].value_counts().max()
del del_time_dict[modal_class]
for x in del_time_dict.keys():
  if x != modal_class:
    del_time_dict[x] = modal_class_value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


In [19]:
numeric_cols = total_data.select_dtypes(include = [int,float]).columns.tolist()
from imblearn.over_sampling import RandomOverSampler
train_resamp,target_resamp = RandomOverSampler(sampling_strategy=del_time_dict,random_state = 1993).fit_resample(train_new[numeric_cols],target_enc)
train_resamp,target_resamp = pd.DataFrame(train_resamp,index = range(train_resamp.shape[0]),columns = numeric_cols),pd.Series(target_resamp,index = range(len(target_resamp)))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [120]:
print(train_data['Delivery_Time_Enc'].value_counts())
print(train_data['Delivery_Time_Enc'].value_counts()/train_data.shape[0])
print(pd.Series(target_resamp).value_counts())

3    7406
4    2665
5     923
1      62
2      20
6      14
0       4
Name: Delivery_Time_Enc, dtype: int64
3    0.667568
4    0.240220
5    0.083198
1    0.005589
2    0.001803
6    0.001262
0    0.000361
Name: Delivery_Time_Enc, dtype: float64
3    7406
6    2665
5    2665
4    2665
2    2665
1    2665
0    2665
dtype: int64


In [121]:
train_resamp.shape

(23396, 106)

In [0]:
xtrain,xval,ytrain,yval = train_test_split(train_resamp,target_resamp,test_size = 0.25,stratify = target_resamp)

In [123]:
clf = RandomForestClassifier(random_state = 1993)
rfe = RFECV(estimator = clf,cv = 5,scoring = 'accuracy',verbose = 2)
rfe_fit = rfe.fit(xtrain,ytrain)

Fitting estimator with 106 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 105 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 104 features.


...

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 35 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 34 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [124]:
#feat_selected = train_resamp[numeric_cols].columns[rfe_fit.support_]
clf = RandomForestClassifier(random_state = 1993)
occ = OneVsRestClassifier(clf)
occ_fit = occ.fit(xtrain,ytrain)
cross_val_score(occ,xtrain,ytrain,cv = 5,scoring = 'accuracy',verbose = 2)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[CV]  ................................................................


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[CV] ................................................. , total=   0.8s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


...

[CV] ................................................. , total=   0.8s
[CV]  ................................................................


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[CV] ................................................. , total=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.9s finished


array([0.89547138, 0.89689547, 0.89888921, 0.897151  , 0.90896119])

In [1]:
occ_preds = occ_fit.predict(xval)
print('Accuracy Score : %s' % accuracy_score(yval,occ_preds) )

NameError: ignored

In [0]:
#### Fitting model on entire train
model_fit = occ.fit(train_resamp,target_resamp)
model_preds = model_fit.predict(test_new[numeric_cols])
model_preds_1 = le_fit.inverse_transform(model_preds)
output_df = pd.DataFrame({'Delivery_Time' : model_preds_1},index = range(len(model_preds)))

In [0]:
output_df.to_excel('Submission_LGBM_1.xlsx',index = False)